We are making clusters out of the embedded data. These clusters represent the data of each individual for categorization purposes

In [ ]:
from utils import save_face_list, extract_frames
from face_extraction import process_image, check_face
from facenet_pytorch import MTCNN
import torch
import cv2
from main_extract_faces import process_movies
from main_unsupervised import find_best_number_of_clusters
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
import os
import shutil
import plots
import augmentation
import main_embed
import cluster_dataframe
import embed

### Video File Handling Code Explanation

This code snippet is responsible for loading and verifying the accessibility of a specific video file using OpenCV (`cv2`).

In [ ]:
#checks your current directory. Using it as a validation that you are in correct directory
os.getcwd()

In [ ]:
MOVIE_FOLDER = "C:\\Users\\ningw\\Desktop\\"  # Change this path to where you stored your movies (don't forget to add \\ at end)

# Input and output file extensions for video files and images
input_extension = '.mp4'
output_extension = '.png'
samples_per_second = 10
padding_x = 10
padding_y = 10
min_confidence = 0.5

# List of movie names for training and testing
MOVIE_TRAINING_LIST = ['New_Kids_ABC', 'New_Kids_Fussballspiel', 
                       'New_Kids_Turbo_Tankstation', 'New_Kids_Nitro_Peter_lemonade_720']
MOVIE_TEST_LIST = ['Test']

# Define current directory variable
current_directory = os.getcwd()  # Dynamically set to the current working directory

# Define paths using the current directory variable
FACES_FOLDER_TRAINING = f"{current_directory}\\face_folder\\"
FACES_FOLDER_TEST = f"{current_directory}\\face_folder_test\\"

OUTPUT_FOLDER_RESULTS_TRAIN = f"{current_directory}\\results\\"
RESULTS_NAME_TRAIN = "results.csv"
RESULTS_CSV_TRAIN = f"{OUTPUT_FOLDER_RESULTS_TRAIN}\\{RESULTS_NAME_TRAIN}"

OUTPUT_FOLDER_RESULTS_TEST = f"{current_directory}\\results_test\\"
RESULTS_NAME_TEST = "test_results.csv"
RESULTS_CSV_TEST = f"{OUTPUT_FOLDER_RESULTS_TEST}\\{RESULTS_NAME_TEST}"

KMEANS_OUTPUT_FOLDER = f"{current_directory}\\KMEANS_OUTPUT\\"
CLUSTER_MODEL_PATH = f"{KMEANS_OUTPUT_FOLDER}\\kmeans.pkl"

# List of all directories
directories = [FACES_FOLDER_TRAINING, FACES_FOLDER_TEST, 
               OUTPUT_FOLDER_RESULTS_TRAIN, OUTPUT_FOLDER_RESULTS_TEST, 
               KMEANS_OUTPUT_FOLDER]

# Check if each directory exists, and create it if it doesn't
for path in directories:
    if not os.path.isdir(path):  # Check if the directory exists
        os.makedirs(path, exist_ok=True)  # Create the directory if it doesn't exist
        print(f"Checked/Created directory: {path}")  # Confirm creation of the directory
    else:
        print(f"Directory already exists: {path}")  # Confirm the directory already exists

### Video File Handling Code Explanation

This code snippet is responsible for loading and verifying the accessibility of a specific video file using OpenCV (`cv2`).

In [ ]:
# OpenCV's VideoCapture is initialized with the path to a video file.
cap = cv2.VideoCapture(MOVIE_FOLDER + MOVIE_TRAINING_LIST[1] + '.mp4')  # Construct the full file path using MOVIE_FOLDER and the second video name in the list.

# Check if the video file has been successfully opened.
if not cap.isOpened():  # .isOpened() returns False if the video file cannot be accessed.
    print("Error: Cannot open video file.")  # Print an error message if the video file cannot be opened.
else:
    print("Video file opened successfully!")  # Print a success message if the video file is opened correctly.

# Release the resources associated with the VideoCapture object.
cap.release()  # It's important to release the VideoCapture object to free resources and avoid memory leaks.

### Calling the `process_movies` Function with Defined Variables

The `process_movies` function is designed to process a list of movies, extract faces from their frames, and save the extracted faces to a specified folder.

In [ ]:
# Call the function using the defined variables
process_movies(
    movie_list=MOVIE_TRAINING_LIST,
    movie_folder=MOVIE_FOLDER,
    faces_folder=FACES_FOLDER_TRAINING,
    input_extension=input_extension,
    output_extension=output_extension,
    samples_per_second=samples_per_second,
    padding_x=padding_x,
    padding_y=padding_y,
    min_confidence=min_confidence
)


### Calling the `embed` Function for the training

The `embed` function is designed to process training face images stored in a folder, compute their embeddings using a neural network, and save the resulting embeddings to a CSV file in the specified output folder. 

In [ ]:
main_embed.embed(FACES_FOLDER_TRAINING, OUTPUT_FOLDER_RESULTS_TRAIN,RESULTS_NAME_TRAIN)

### Choosing the Optimal Number of Clusters: Calinski-Harabasz vs. Silhouette

In our clustering process, we evaluated two different methods to determine the optimal number of clusters: **Silhouette Score** and **Calinski-Harabasz Score**. We opted for the Calinski since it performed better


In [ ]:
# Use the find_best_number_of_clusters function to determine the optimal number of clusters 
# based on silhouette and Calinski-Harabasz scores from the training results.
best_clusters_silhouette, best_clusters_calinski = find_best_number_of_clusters(RESULTS_CSV_TRAIN)

# We had to choose between Silhouette or Calinski. We opted to go for Calinski score for our cluster choice. 
# We found that running with 7 clusters leads to too many overlaps with identities
cluster_choice = best_clusters_calinski


### Processing testing data

We do the same processing for our testing data

In [ ]:
# Function call for testing movies
process_movies(
    movie_list=MOVIE_TEST_LIST,
    movie_folder=MOVIE_FOLDER,
    faces_folder=FACES_FOLDER_TEST,
    input_extension=input_extension,
    output_extension=output_extension,
    samples_per_second=samples_per_second,
    padding_x=padding_x,
    padding_y=padding_y,
    min_confidence=min_confidence
)


In [ ]:
main_embed.embed(FACES_FOLDER_TEST,OUTPUT_FOLDER_RESULTS_TEST,RESULTS_NAME_TEST)

# Exploring the Outcomes

After successfully processing the embeddings and organizing images based on clusters and outliers, we are now starting to explore the results of our analysis. 


In [ ]:
# Load training embeddings from the CSV file
train_embeddings_df = pd.read_csv(RESULTS_CSV_TRAIN)  # Read embeddings for training faces into a DataFrame.
trains_embeddings = train_embeddings_df.T.values  # Transpose the DataFrame and convert it to NumPy array format.

# Load testing embeddings from the CSV file
test_embeddings_df = pd.read_csv(RESULTS_CSV_TEST)  # Read embeddings for test faces into a DataFrame.
test_embeddings = test_embeddings_df.T.values  # Transpose the DataFrame and convert it to NumPy array format.

# Perform KMeans clustering on the training embeddings
kmeans = KMeans(random_state=0, n_clusters=cluster_choice).fit(trains_embeddings)  # Fit KMeans model using the training embeddings.
labels = kmeans.labels_  # Assign cluster labels to the training embeddings.

# Calculate distances from each point to its closest cluster center
distances_train = pairwise_distances_argmin_min(trains_embeddings, kmeans.cluster_centers_)[1]  # Compute distances from each embedding to its cluster center.

# Determine the 85th percentile distance as the threshold for outliers
threshold_distance_85 = np.percentile(distances_train, 85)  # Set the threshold based on the 85th percentile of distances.

# Plot histogram of distances and the 85th percentile threshold
plots.plot_histplot_percentile(distances_train, threshold_distance_85)  # Generate a histogram to visualize distances.

# Remove embeddings considered outliers based on the threshold
for i, file_name in enumerate(train_embeddings_df.columns):  # Iterate over each column (embedding) in the DataFrame.
    if distances_train[i] > threshold_distance_85:  # Check if the distance exceeds the outlier threshold.
        train_embeddings_df.drop(file_name, axis='columns', inplace=True)  # Drop the outlier column from the DataFrame.

# Recompute embeddings array after removing outliers
trains_embeddings = train_embeddings_df.T.values  # Update the embeddings array after removing outliers.

# Re-run KMeans clustering with a higher maximum iteration limit
kmeans = KMeans(random_state=0, n_clusters=cluster_choice, max_iter=600).fit(trains_embeddings)  # Fit the updated training embeddings to KMeans.
labels = kmeans.labels_  # Get updated cluster labels after re-running KMeans.

# Organize images into cluster folders and outlier folders
for i, file_name in enumerate(train_embeddings_df.columns):  # Iterate through each embedding in the DataFrame.
    src_path = os.path.join(FACES_FOLDER_TRAINING, file_name)  # Define the source path of the image.
    
    # Assign the image to the "outliers" folder or its respective cluster folder
    if distances_train[i] > threshold_distance_85:  # Check if the image is an outlier.
        dst_path = os.path.join(KMEANS_OUTPUT_FOLDER, 'outliers', file_name)  # Set destination to "outliers" folder.
    else:
        dst_path = os.path.join(KMEANS_OUTPUT_FOLDER, f'cluster_{labels[i]}', file_name)  # Set destination to the corresponding cluster folder.

    # Ensure the destination directory exists before saving the image
    os.makedirs(os.path.dirname(dst_path), exist_ok=True)  # Create the directory if it does not exist.
    shutil.copy(src_path, dst_path)  # Copy the image to the destination path.



In [ ]:
# Define a dictionary to store DataFrames.
df_dict = {}

# Define a list of thresholds to evaluate.
thresholds = [0.80, 0.85, 0.9, 0.95]

# Iterate through each threshold value.
for threshold in thresholds:
    # Calculate the threshold distance for the current percentile of the training distances.
    threshold_distance = np.percentile(distances_train, threshold * 100)  
    
    # Process test videos to create a DataFrame of clusters using the calculated threshold distance.
    clusterdf = cluster_dataframe.process_videos_to_dataframe(
        MOVIE_TEST_LIST, MOVIE_FOLDER, kmeans, threshold_distance_85=threshold_distance
    )
    
    # Store the DataFrame in the dictionary with the threshold as the key.
    df_dict[f"df_{threshold}"] = clusterdf

    # Plot the presence of clusters in the sorted DataFrame to visualize their distribution.
    plots.plot_cluster_presence(clusterdf.sort_index(ascending=False))

# At the end of the loop, you will have separate DataFrame variables like df_0.80, df_0.85, etc.


### DataFrames for Heatmaps

Listed below are the DataFrames generated from the earlier loop. These DataFrames are used to create the heatmaps. For your convenience, we've included them here, allowing you to explore the results if desired.

In [ ]:
df_dict["df_80"]

In [ ]:
df_dict["df_85"]

In [ ]:
df_dict["df_90"]

In [ ]:
df_dict["df_95"]

# Performing Image Augmentation to Assess Impact

Image augmentation is a technique used to artificially adjust the dataset by applying transformations such as flipping, rotation, noise addition, or distortion. This helps evaluate the robustness of clustering and embedding models when faced with varied and augmented data.

In [ ]:
# Apply image augmentations to the test face images stored in FACES_FOLDER_TEST.
# The augmented images are saved in a designated folder named 'AUGMENTED'.
augmentation.process_images_with_augmentations(FACES_FOLDER_TEST, 'AUGMENTED')

In [ ]:
# Iterate through all folders in the 'AUGMENTED' directory
for folder_name in os.listdir('AUGMENTED'):  # List all items in the 'AUGMENTED' directory.
    folder_path = os.path.join('AUGMENTED', folder_name, '')  # Construct the full path to the current folder.

    if os.path.isdir(folder_path):  # Check if the current item is a directory.
        # Dynamically construct paths for processing the folder
        AUGMENTED_FOLDERS = folder_path  # Set the path of the current folder as the augmented data folder.
        RESULT_NAME_TRAIN = f'embed_{folder_name}.csv'  # Dynamically create a CSV file name based on the folder name.

        # Debugging: Print paths to verify correctness
        print(f"Processing folder: {AUGMENTED_FOLDERS}")  # Print the path of the augmented folder being processed.
        print(f"Output folder: {OUTPUT_FOLDER_RESULTS_TEST}")  # Print the path where the results will be saved.
        print(f"Result file: {RESULT_NAME_TRAIN}")  # Print the name of the result file.

        # Call embedding function to process the augmented data
        main_embed.embed(AUGMENTED_FOLDERS, OUTPUT_FOLDER_RESULTS_TEST, RESULT_NAME_TRAIN)
        # The embedding function processes images in the augmented folder,
        # computes embeddings, and saves the results in the specified output folder as a CSV file.


## Saving Outcomes and Plots

Processed results for different thresholds are saved as `.csv` files in the `test_results` folder. Thresholds like `0.80`, `0.85`, `0.90`, and `0.95` are used to analyze clustering sensitivity. Visual plots are created to show actor presence across clusters, aiding in understanding data patterns and threshold impact.



In [ ]:
# Ensure the test_results directory exists
os.makedirs("test_results", exist_ok=True)  
# This ensures the directory "test_results" is created if it does not already exist,
# allowing results to be saved without errors.

thresholds = [0.80, 0.85, 0.9, 0.95]
for threshold in thresholds:
    # Calculate the threshold distance for the current percentile
    threshold_distance = np.percentile(distances_train, threshold * 100)
    
    # Process the test embeddings with the current threshold
    results_df = cluster_dataframe.process_test_embeddings(
        test_embeddings=test_embeddings,
        test_embeddings_df=test_embeddings_df,
        kmeans=kmeans,
        output_folder='recognition-output',
        face_folder_test='face_folder_test',
        threshold=threshold_distance,
        results_output_path = f'test_results\\test_results_{threshold}.csv',
    )
    
    # Plot actor presence
    plots.plot_actor_presence(results_df, 'Cluster', cluster_choice,f'test_results_{threshold}.csv')


## Processing Augmented Results

Augmented results are saved as `.csv` files in the `distorted_results` directory. Each file in the `OUTPUT_FOLDER_RESULTS_TEST` directory is processed, converting embeddings for clustering using a pre-trained KMeans model and a distance threshold (`threshold_distance_85`). Results are analyzed and plotted to visualize actor presence across clusters, helping assess the robustness of clustering models against augmented data.


In [ ]:
# Ensure the distorted_result directory exists
os.makedirs("distorted_results", exist_ok=True)  
# This ensures the directory "distorted_result" is created if it does not already exist,
# allowing results to be saved without errors.

# Iterate through all files in the OUTPUT_FOLDER_RESULTS_TEST directory
for file_name in os.listdir(OUTPUT_FOLDER_RESULTS_TEST):  
    file_path = os.path.join(OUTPUT_FOLDER_RESULTS_TEST, file_name)  # Construct the full file path.

    # Check if the current file is a CSV file
    if file_name.endswith('.csv') and os.path.isfile(file_path):  
        print(f"Processing file: {file_name}")  # Debugging: Print the name of the file being processed.

        # Read the embeddings from the CSV file
        test_embeddings_df = pd.read_csv(file_path)  # Load the embeddings into a Pandas DataFrame.
        test_embeddings = test_embeddings_df.T.values  # Convert the DataFrame to a NumPy array for processing.
        print(test_embeddings_df.columns)  # Debugging: Print the column names of the embeddings.

        # Process the embeddings and generate results
        results_df = cluster_dataframe.process_test_embeddings(
            test_embeddings=test_embeddings,  # Array of test embedding vectors.
            test_embeddings_df=test_embeddings_df,  # DataFrame containing metadata of test embeddings.
            kmeans=kmeans,  # Pre-trained KMeans model for clustering.
            output_folder='recognition-output',  # Directory where clustering results will be saved.
            face_folder_test="face_folder_test",  # Directory containing test face images.
            threshold=threshold_distance_85,  # Distance threshold for determining cluster assignment.
            results_output_path=f"distorted_results\\{file_name}",  # File path for saving results.
        )
        # The function processes test embeddings, assigns clusters, and saves results in the specified directory.

        # Plot actor presence based on cluster assignment
        plots.plot_actor_presence(results_df, 'Cluster', cluster_choice, file_name)  
        # Generate a histogram showing actor presence in clusters using the processed results.
